In [ ]:
import scipy.constants
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import scipy as sp
from scipy.optimize import minimize
from matplotlib.patches import Ellipse

**Plots**

In [ ]:
# Plot an array as a line plot
# x are the values on the x-axis, y on the y axis
# np.arange makes a list with values from min to max with a stepsize of s

x  = np.arange(min, max, s)
plt.figure(1)
plt.plot(x,y)
plt.show()

# Plot circles
c1 = plt.Circle((0,0), 15, alpha=0.5)
c2 = plt.Circle((0,20), 15, alpha=0.5)
c3 = plt.Circle((20,10), 15, alpha=0.5)
fig,ax=plt.subplots()
plt.xlim(-16,36) # set x-limit of figure
plt.ylim(-16,36) # set y-limit of figure

ax.add_artist(c1)
ax.add_artist(c2)
ax.add_artist(c3)

plt.show()

# Plot Histogram from array
plt.hist(array, bins=10)
plt.show()

# Plot ellipse
# position is the x,y position in the plot
e = Ellipse(position, width = stdsx, height=stdsy)
plt.gca().add_artist(e) 
e.set_alpha(0.2) # set opacity (transparancy) of the ellipse
e.set_facecolor('g') # g = green, r = red, etc
plt.title('Title of the plot')
plt.show()

**Chi squared**

In [ ]:
def chisquared(routers, assumedClient, actualClient, sigma):
    x,y,z = assumed
    xreal, yreal, zreal = actual
    Chi = 0
    for xi,yi,zi in datapoint:
        Si = getExpected(x,y,xi,yi,zi-z)
        Pri = getMeasured(xreal,yreal,xi,yi,zreal-zi)
        Chi += math.pow(Si-Pri,2)/math.pow(sigma,2)
    return Chi

routers = [[0, 0, 3],[ 0, 20, 3],[20, 0, 3],[20, 20, 3]]
actualClient = [5,5,1]
X = np.arange(-5,16,1)
Chis = [[x, estimatePositionWithoutNoise(routers, [x,5,1], actualClient, 1, 20, 0)] for x in X]

Chis = np.matrix(Chis)
plt.plot(Chis[:,0],Chis[:,1])
plt.show()

**Minimization**

In [ ]:
def G(x):
    # Some function is used as a place holder to give multiple parameters to minimization
    # Returns function that returns a score (e.g. chi squared)
    return actualFunction(x[0],x[1])

results = []
Chis = []

for i in range(0,1000):
    minimized = minimize(G,[x,y], method='Powell')
    results.append(minimized.x) # optimized parameters
    Chis.append(minimized.fun) # score of the parameter

**Uncertainty**

Sigma is the estimated measurement uncertainty

In [ ]:
def estimateUncertainty(group, position):
    B11 = B12 = B13 = B21 = B22 = B23 = B31 = B32 = B33 = 0
    for packet in group:
        routerLocation = routers[packet['droneId']]
        x = routerLocation[0]
        y = routerLocation[1]
        dFdx = # derivation of function to x (e.g. Friis equation)
        dFdy = # derivation of function to y (e.g. Friis equation)
        '''
        dFdx = (40*(position[0] - x))/(math.log(10)*((position[0]-x)**2 + (position[1]-y)**2 + Z**2))
        dFdy = (40*(position[1] - y))/(math.log(10)*((position[0]-x)**2 + (position[1]-y)**2 + Z**2))
        '''
        dFdPt = 1
        B11 += (dFdx**2 * 1/(sigma**2))
        B22 += (dFdy**2 * 1/(sigma**2))
        B12 += (dFdx * dFdy * 1/(sigma**2))
        B21 = B12
        B13 += (dFdx * dFdPt * 1/(sigma**2))
        B31 = B13
        B23 += (dFdy * dFdPt * 1/(sigma**2))
        B32 = B23
        B33 += (dFdPt**2 * 1/(sigma**2))
    
    B = np.matrix([[B11,B12,B13],[B21,B22,B23],[B31,B32,B33]])
    
    vary, varx, varPt = np.diagonal(np.linalg.inv(B))
    
    stdsx.append(np.sqrt(varx))
    stdsy.append(np.sqrt(vary))
    stdsPt.append(np.sqrt(varPt))       


**Pull Distribution**

**Read csv file**